In [2]:
import requests
import json
import time

BASE_URL = "http://localhost:8080"

def print_response(resp):
    print("Status:", resp.status_code)
    try:
        print("Response:", json.dumps(resp.json(), indent=2, ensure_ascii=False))
    except Exception:
        print("Response (raw):", resp.text)
    print("="*40)

# Iniciar sesión
def login(email, password):
    url = f"{BASE_URL}/api/auth/login"
    data = {"email": email, "password": password}
    resp = requests.post(url, json=data)
    print("Login:", email)
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Crear caso (aviso) – acción exclusiva de usuario
def create_case(creador_id, titulo, descripcion, categoria, imagenR, importancia):
    url = f"{BASE_URL}/api/casos/crear"
    payload = {
        "titulo": titulo,
        "descripcion": descripcion,
        "categoria": categoria,
        "imagenR": imagenR,
        "importancia": importancia,  # Valores: ALTO, MEDIO o BAJO
        "creadorId": creador_id
    }
    resp = requests.post(url, json=payload)
    print("Crear Caso:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Consultar casos abiertos (para superhéroes)
def get_open_cases():
    url = f"{BASE_URL}/api/casos/abiertos"
    resp = requests.get(url)
    print("Lista de casos abiertos:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# El superhéroe acepta un caso y se crea (debería) un chat
def accept_case(aviso_id, heroe_id):
    url = f"{BASE_URL}/api/casos/{aviso_id}/aceptar"
    payload = {"heroeId": heroe_id}
    resp = requests.post(url, json=payload)
    print(f"Aceptar Caso {aviso_id} por heroe {heroe_id}:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Obtener lista de chats para una cuenta (usuario o superhéroe)
def get_chats(cuenta_id):
    url = f"{BASE_URL}/api/chats/user/{cuenta_id}"
    resp = requests.get(url)
    print(f"Chats para cuenta id {cuenta_id}:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Enviar mensaje en un chat
def send_message(chat_id, autor_id, texto):
    url = f"{BASE_URL}/api/chats/{chat_id}/mensaje"
    payload = {"autorId": autor_id, "texto": texto}
    resp = requests.post(url, json=payload)
    print(f"Enviar mensaje en Chat {chat_id} desde autor {autor_id}:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Función principal para simular todo el flujo de conversación
if __name__ == "__main__":
    # Datos de prueba basados en DefaultDataInitializerService:
    # Usuario: user1@example.com / pass123
    # Superhéroe: ironman@example.com / pass123
    usuario = login("user1@example.com", "pass123")
    heroe = login("ironman@example.com", "pass123")
    if not usuario or not heroe:
        print("Error en el login. Verifica que las credenciales sean correctas y que los datos por defecto se hayan inicializado.")
        exit(1)
    
    # El usuario crea un caso (aviso)
    nuevo_caso = create_case(
        creador_id=usuario["id"],
        titulo="Urgente: Necesito ayuda",
        descripcion="Se requiere asistencia inmediata en el área central.",
        categoria="Emergencia",
        imagenR="emergencia.jpg",
        importancia="ALTO"
    )
    if not nuevo_caso:
        print("Error al crear caso.")
        exit(1)
    
    # Pequeña pausa para asegurar que el caso se haya guardado
    time.sleep(1)
    
    # El superhéroe consulta casos abiertos y selecciona uno para aceptar
    casos_abiertos = get_open_cases()
    if not casos_abiertos:
        print("No se encontraron casos abiertos.")
        exit(1)
    caso_a_aceptar = casos_abiertos[0]
    aviso_id = caso_a_aceptar["id"]
    print(f"Caso a aceptar: {aviso_id}")
    
    respuesta_aceptar = accept_case(aviso_id, heroe["id"])
    chat_id = respuesta_aceptar.get("chatId", None)
    if chat_id is None:
        print("No se creó chat al aceptar el caso.")
        exit(1)
    print(f"Chat creado con id: {chat_id}")
    
    # Pausa antes de comenzar la conversación
    time.sleep(1)
    
    # Se simula la conversación:
    # 1. Usuario envía un mensaje
    send_message(chat_id, usuario["id"], "Hola, he creado el caso, estoy a la espera de tu llegada.")
    time.sleep(1)
    # 2. Superhéroe responde
    send_message(chat_id, heroe["id"], "Hola, soy Iron Man. Llegaré lo más pronto posible.")
    time.sleep(1)
    # 3. Usuario envía otro mensaje
    send_message(chat_id, usuario["id"], "Perfecto, espero tu llegada. Por favor, mantente en contacto.")
    time.sleep(1)
    # 4. Superhéroe responde nuevamente
    send_message(chat_id, heroe["id"], "Claro, te estaré informando en todo momento.")
    
    # Consultar el chat del usuario para verificar todos los mensajes enviados
    chats_usuario = get_chats(usuario["id"])
    print("Conversación en el primer chat encontrado:")
    if chats_usuario:
        chat_actual = chats_usuario[0]
        print("Chat ID:", chat_actual["id"])
        # Se asume que la respuesta incluye una lista de mensajes
        if "mensajes" in chat_actual:
            for msg in chat_actual["mensajes"]:
                print(f'{msg.get("fechaCreacion", "Sin fecha")} - {msg.get("autor", {}).get("nombre", "Desconocido")}: {msg.get("texto")}')
        else:
            print("No se encontraron mensajes en el chat.")
    else:
        print("No se encontraron chats para el usuario.")

Login: user1@example.com
Status: 200
Response: {
  "imagenPerfil": "user1.jpg",
  "id": 9,
  "nombre": "Usuario Uno",
  "email": "user1@example.com",
  "rol": "usuario"
}
Login: ironman@example.com
Status: 200
Response: {
  "imagenPerfil": "iron_man.jpg",
  "id": 5,
  "nombre": "Iron Man",
  "email": "ironman@example.com",
  "rol": "superheroe"
}
Crear Caso:
Status: 200
Response: {
  "id": 0,
  "creador": {
    "id": 9,
    "nombre": "Usuario Uno",
    "email": "user1@example.com",
    "password": "pass123",
    "imagenPerfil": "user1.jpg",
    "notificaciones": [],
    "fechaCreacion": "2025-03-04T02:56:08.166157"
  },
  "titulo": "Urgente: Necesito ayuda",
  "descripcion": "Se requiere asistencia inmediata en el área central.",
  "categoria": "Emergencia",
  "imagenR": "emergencia.jpg",
  "fechaCreacion": "2025-03-04T20:23:58.2204457",
  "importancia": "ALTO",
  "disponible": true
}
Lista de casos abiertos:
Status: 200
Response: [
  {
    "id": 45,
    "creador": {
      "id": 9,
   

In [1]:
import requests
import json
import time

BASE_URL = "http://localhost:8080"


# Iniciar sesión
def login(email, password):
    url = f"{BASE_URL}/api/auth/login"
    data = {"email": email, "password": password}
    resp = requests.post(url, json=data)
    print("Login:", email)
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Crear caso (aviso) – acción exclusiva para usuarios
def create_case(creador_id, titulo, descripcion, categoria, imagenR, importancia):
    url = f"{BASE_URL}/api/casos/crear"
    payload = {
        "titulo": titulo,
        "descripcion": descripcion,
        "categoria": categoria,
        "imagenR": imagenR,
        "importancia": importancia,  # Valores: ALTO, MEDIO o BAJO
        "creadorId": creador_id
    }
    resp = requests.post(url, json=payload)
    print("Crear Caso:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Consultar casos abiertos (para superhéroes)
def get_open_cases():
    url = f"{BASE_URL}/api/casos/abiertos"
    resp = requests.get(url)
    print("Lista de casos abiertos:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# El superhéroe acepta un caso (lo que debería crear un chat)
def accept_case(aviso_id, heroe_id):
    url = f"{BASE_URL}/api/casos/{aviso_id}/aceptar"
    payload = {"heroeId": heroe_id}
    resp = requests.post(url, json=payload)
    print(f"Aceptar Caso {aviso_id} por heroe {heroe_id}:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

# Obtener chats de una cuenta
def get_chats(cuenta_id):
    url = f"{BASE_URL}/api/chats/user/{cuenta_id}"
    resp = requests.get(url)
    print(f"Chats para cuenta id {cuenta_id}:")
    
    try:
        if resp.status_code == 200:
            json_data = resp.json()
            print_response(resp)
            return json_data
        else:
            print(f"Error en la respuesta: {resp.status_code}")
            print(f"Respuesta truncada: {resp.text[:500]}...")
            print("="*40)
            return []
    except json.JSONDecodeError as e:
        print(f"Error al decodificar JSON: {str(e)}")
        print(f"Respuesta truncada: {resp.text[:500]}...")
        print("="*40)
        return []

def print_response(resp):
    print("Status:", resp.status_code)
    try:
        if resp.text.strip():  # Verificar que la respuesta no esté vacía
            print("Response:", json.dumps(resp.json(), indent=2, ensure_ascii=False))
        else:
            print("Response: [empty]")
    except json.JSONDecodeError:
        print(f"Response (raw, truncated): {resp.text[:500]}...")
    print("="*40)

# Enviar mensaje en un chat
def send_message(chat_id, autor_id, texto):
    url = f"{BASE_URL}/api/chats/{chat_id}/mensaje"
    payload = {"autorId": autor_id, "texto": texto}
    resp = requests.post(url, json=payload)
    print(f"Enviar mensaje en Chat {chat_id} desde autor {autor_id}:")
    print_response(resp)
    return resp.json() if resp.status_code == 200 else None

if __name__ == "__main__":
    # 1. Login: usa los datos creados en DefaultDataInitializerService
    # Usuario: user1@example.com / pass123
    # Superhéroe: ironman@example.com / pass123
    usuario = login("user1@example.com", "pass123")
    heroe = login("ironman@example.com", "pass123")
    if not usuario or not heroe:
        print("Error en login. Verifica las credenciales y que los datos por defecto se hayan inicializado.")
        exit(1)
    
    # 2. Crear caso como usuario
    nuevo_caso = create_case(
        creador_id=usuario["id"],
        titulo="Urgente: Necesito ayuda",
        descripcion="Se requiere asistencia inmediata en el área central.",
        categoria="Emergencia",
        imagenR="emergencia.jpg",
        importancia="ALTO"
    )
    if not nuevo_caso:
        print("Error al crear caso.")
        exit(1)
    
    # Verifica si el id del caso es correcto
    if nuevo_caso.get("id", 0) == 0:
        print("El caso se creó con id 0. Revisa la lógica de persistencia en el CaseService o la entidad Aviso.")
    
    # 3. El superhéroe consulta casos abiertos
    time.sleep(1)
    casos_abiertos = get_open_cases()
    if not casos_abiertos:
        print("No se encontraron casos abiertos.")
        exit(1)
    
    caso_a_aceptar = casos_abiertos[0]
    aviso_id = caso_a_aceptar["id"]
    print(f"Caso a aceptar: {aviso_id}")
    
    # 4. El superhéroe acepta el caso y se crea un chat
    respuesta_aceptar = accept_case(aviso_id, heroe["id"])
    chat_id = respuesta_aceptar.get("chatId", None)
    if chat_id is None:
        print("No se creó chat al aceptar el caso.")
        exit(1)
    print(f"Chat creado con id: {chat_id}")
    
    # 5. Pausa y comenzar la conversación
    time.sleep(1)
    
    # Usuario envía un mensaje
    send_message(chat_id, usuario["id"], "Hola, he creado el caso, estoy a la espera de tu llegada.")
    time.sleep(1)
    # Superhéroe responde
    send_message(chat_id, heroe["id"], "Hola, soy Iron Man. Llegaré lo más pronto posible.")
    time.sleep(1)
    # Usuario envía un segundo mensaje
    send_message(chat_id, usuario["id"], "Perfecto, espero tu llegada. Por favor, mantente en contacto.")
    time.sleep(1)
    # Superhéroe confirma
    send_message(chat_id, heroe["id"], "Claro, te iré informando en todo momento.")
    
    # 6. Consultar el chat del usuario para verificar la conversación
    time.sleep(1)
    chats_usuario = get_chats(usuario["id"])
    print("Conversación en el primer chat encontrado:")
    if chats_usuario:
        chat_actual = chats_usuario[0]
        print("Chat ID:", chat_actual["id"])
        if "mensajes" in chat_actual:
            for msg in chat_actual["mensajes"]:
                autor = msg.get("autor", {})
                print(f'{msg.get("fechaCreacion", "Sin fecha")} - {autor.get("nombre", "Desconocido")}: {msg.get("texto")}')
        else:
            print("No se encontraron mensajes en el chat.")
    else:
        print("No se encontraron chats para el usuario.")

Login: user1@example.com
Status: 200
Response: {
  "imagenPerfil": "user1.jpg",
  "id": 9,
  "nombre": "Usuario Uno",
  "email": "user1@example.com",
  "rol": "usuario"
}
Login: ironman@example.com
Status: 200
Response: {
  "imagenPerfil": "iron_man.jpg",
  "id": 5,
  "nombre": "Iron Man",
  "email": "ironman@example.com",
  "rol": "superheroe"
}
Crear Caso:
Status: 200
Response: {
  "id": 0,
  "creador": {
    "id": 9,
    "nombre": "Usuario Uno",
    "email": "user1@example.com",
    "password": "pass123",
    "imagenPerfil": "user1.jpg",
    "notificaciones": [],
    "fechaCreacion": "2025-03-04T02:56:08.166157"
  },
  "titulo": "Urgente: Necesito ayuda",
  "descripcion": "Se requiere asistencia inmediata en el área central.",
  "categoria": "Emergencia",
  "imagenR": "emergencia.jpg",
  "fechaCreacion": "2025-03-04T20:23:35.4926205",
  "importancia": "ALTO",
  "disponible": true
}
El caso se creó con id 0. Revisa la lógica de persistencia en el CaseService o la entidad Aviso.
Lista